#### Problem 1
#### Inspect the dataset titled lab01_dataset_1.csv which has a mixture of numerical and categorical data. Your task will be to write a function my_ID3( ) which can create a decision tree for the given dataset using the ID3 algorithm. However, before doing that, you will be have to perform some data processing tasks. Here are all the required tasks in order –

In [23]:
import pandas as pd
import numpy as np
import pprint
from sklearn.preprocessing import OrdinalEncoder
eps = np.finfo(float).eps
print(eps)

2.220446049250313e-16


In [24]:
data = pd.read_csv("lab01_dataset_1.csv")
data.head()

,Mood,Effort,Score,Output
0,Happy,Low,35,Yes
1,Happy,Medium,91,No
2,Happy,High,52,No
3,Neutral,Low,83,No
4,Neutral,Medium,48,No


##### 1. ID3 cannot handle continuous numerical data. Perform necessary operations to handle all continuous-valued attributes. Do not forget to show the output i.e., the updated dataset after handling continuous-valued attributes. (2 marks

In [25]:
data.tail()

,Mood,Effort,Score,Output
14,Neutral,High,50,No
15,Sad,Low,69,No
16,Sad,Medium,70,Yes
17,Sad,High,95,No
18,Sad,Low,80,Yes


In [26]:
print(data.columns)
print(data.shape)
print(data.dtypes)

Index(['Mood', 'Effort', 'Score', 'Output'], dtype='object')
(19, 4)
Mood      object
Effort    object
Score      int64
Output    object
dtype: object


In [27]:
# sort the data by score
def sortDataFrameByScore(data):
    data = data.sort_values(by='Score', ignore_index=True)
    data.head()
    return data

data = sortDataFrameByScore(data)
data

,Mood,Effort,Score,Output
0,Happy,Low,35,Yes
1,Happy,High,39,Yes
2,Sad,Low,40,Yes
3,Happy,Low,44,Yes
4,Neutral,Medium,48,No
5,Neutral,High,50,No
6,Happy,High,52,No
7,Neutral,High,61,No
8,Happy,Low,66,No
9,Sad,Low,69,No


In [28]:
def splitPoint(data):
    attributeDataFrame = pd.DataFrame({ 'Score' : data['Score'], 'Output' : data['Output']})
    attributeDataFrame.reindex(np.arange(len(attributeDataFrame['Score'].index)))

    outputvalue = attributeDataFrame['Output'][0]

    splitPointDict = {}

    for i in range(1, len(attributeDataFrame['Score'])):
        if(attributeDataFrame['Output'][i] != outputvalue):
            mean = (attributeDataFrame['Score'][i-1] + attributeDataFrame['Score'][i])/2
            splitPointDict[i-1] = mean
            outputvalue = attributeDataFrame['Output'][i]
        
    pprint.pprint(splitPointDict)
    return attributeDataFrame, splitPointDict
attributeDataFrame, splitPoint = splitPoint(data)


{3: 46.0, 9: 69.5, 13: 81.5}


In [29]:
for i in splitPoint:
    split = splitPoint[i]
    print(splitPoint[i])
    ltYes = 0
    ltNo = 0
    gtYes = 0
    gtNo = 0

    for j in range(0, len(attributeDataFrame['Score'])):
        if attributeDataFrame['Score'][j] < split and attributeDataFrame['Output'][j] == 'Yes':
            ltYes+=1
        if attributeDataFrame['Score'][j] < split and attributeDataFrame['Output'][j] == 'No':
            ltNo+=1

        if attributeDataFrame['Score'][j] >= split and attributeDataFrame['Output'][j] == 'Yes':
            gtYes+=1
        if attributeDataFrame['Score'][j] >= split and attributeDataFrame['Output'][j] == 'No':
            gtNo+=1

    print(ltYes , " | " , ltNo  , " | " , gtYes , " | " , gtNo)

46.0
4  |  0  |  4  |  11
69.5
4  |  6  |  4  |  5
81.5
8  |  6  |  0  |  5


In [30]:
columnsList = {}

for i in splitPoint:
    split = splitPoint[i]
    # "Score_" + str(split)
    scoreList=[]
    for j in range(0, len(attributeDataFrame['Score'])):
        if attributeDataFrame['Score'][j] < split:
            scoreList.append(1) # ("Score_" + str(split) + "True")
        else: 
            scoreList.append(0) # ("Score_" + str(split) + "False")

    columnsList["Score_" + str(split)] = scoreList

pprint.pprint(columnsList)

{'Score_46.0': [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'Score_69.5': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'Score_81.5': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]}


In [31]:
# data['Score_81.5'] = columnsList['Score_81.5']
# data['Score_46.0'] = columnsList['Score_46.0']
# data['Score_69.5'] = columnsList['Score_69.5']

In [32]:
# feature Score is dropped
data = pd.DataFrame({
    'Mood' : data['Mood'], 
    'Effort' : data['Effort'], 
    'Score_46.0' : columnsList['Score_46.0'], 
    'Score_69.5' : columnsList['Score_69.5'], 
    'Score_81.5' : columnsList['Score_81.5'],
    'Output' : data['Output']
})

data.head()

,Mood,Effort,Score_46.0,Score_69.5,Score_81.5,Output
0,Happy,Low,1,1,1,Yes
1,Happy,High,1,1,1,Yes
2,Sad,Low,1,1,1,Yes
3,Happy,Low,1,1,1,Yes
4,Neutral,Medium,0,1,1,No


In [33]:
moodUniqueValues = list(np.asarray(data['Mood'].unique()))
encoderMood = OrdinalEncoder(categories=[list(moodUniqueValues)])
# data['Mood'] = encoderMood.fit_transform(data[['Mood']])

effortUniqueValues = list(np.asarray(data['Effort'].unique()))
encoderEffort = OrdinalEncoder(categories=[list(effortUniqueValues)])
# data['Effort'] = encoderEffort.fit_transform(data[['Effort']])

outputUniqueValues = list(np.asarray(data['Output'].unique()))
encoderOutput = OrdinalEncoder(categories=[list(outputUniqueValues)])
# data['Output'] = encoderOutput.fit_transform(data[['Output']])

print(moodUniqueValues)
print(effortUniqueValues)
print(outputUniqueValues)

data.head()

['Happy', 'Sad', 'Neutral']
['Low', 'High', 'Medium']
['Yes', 'No']


,Mood,Effort,Score_46.0,Score_69.5,Score_81.5,Output
0,Happy,Low,1,1,1,Yes
1,Happy,High,1,1,1,Yes
2,Sad,Low,1,1,1,Yes
3,Happy,Low,1,1,1,Yes
4,Neutral,Medium,0,1,1,No


#### 2. Next, you will have to ensure the newly obtained dataset is optimal and free of errors. Take appropriate actions based on the outcomes.

- a. Check if the dataset has any missing values. (1 mark)

In [36]:
missingValues = data.isnull()
print(missingValues)

     Mood  Effort  Score_46.0  Score_69.5  Score_81.5  Output
0   False   False       False       False       False   False
1   False   False       False       False       False   False
2   False   False       False       False       False   False
3   False   False       False       False       False   False
4   False   False       False       False       False   False
5   False   False       False       False       False   False
6   False   False       False       False       False   False
7   False   False       False       False       False   False
8   False   False       False       False       False   False
9   False   False       False       False       False   False
10  False   False       False       False       False   False
11  False   False       False       False       False   False
12  False   False       False       False       False   False
13  False   False       False       False       False   False
14  False   False       False       False       False   False
15  Fals

In [37]:
for i in data.columns:
    print("Any missing values in " ,i, ":" , data[i].isnull().any())

Any missing values in  Mood : False
Any missing values in  Effort : False
Any missing values in  Score_46.0 : False
Any missing values in  Score_69.5 : False
Any missing values in  Score_81.5 : False
Any missing values in  Output : False


- b. Check if the dataset has any redundant or repeated input sample. (1 mark)


In [39]:
# Checking for duplicates

dataFrame = pd.DataFrame(data)
duplicatedRows = dataFrame[dataFrame.duplicated()]
print(duplicatedRows)

       Mood  Effort  Score_46.0  Score_69.5  Score_81.5 Output
3     Happy     Low           1           1           1    Yes
7   Neutral    High           0           1           1     No
16    Happy  Medium           0           0           0     No


#### c. Check if the dataset has any contradicting <input, output> pairs. (1 mark)

In [40]:
dataFrame = pd.DataFrame(data)
print(list(data.columns))

contradicting = dataFrame[dataFrame.duplicated(subset=list(data.columns), keep=False)]

print("Original X dataframe : ")
print(dataFrame)
print("\nContradicting <input, output> Pairs:")
print(contradicting)

['Mood', 'Effort', 'Score_46.0', 'Score_69.5', 'Score_81.5', 'Output']
Original X dataframe : 
       Mood  Effort  Score_46.0  Score_69.5  Score_81.5 Output
0     Happy     Low           1           1           1    Yes
1     Happy    High           1           1           1    Yes
2       Sad     Low           1           1           1    Yes
3     Happy     Low           1           1           1    Yes
4   Neutral  Medium           0           1           1     No
5   Neutral    High           0           1           1     No
6     Happy    High           0           1           1     No
7   Neutral    High           0           1           1     No
8     Happy     Low           0           1           1     No
9       Sad     Low           0           1           1     No
10      Sad  Medium           0           0           1    Yes
11      Sad    High           0           0           1    Yes
12  Neutral  Medium           0           0           1    Yes
13      Sad     Low    

#### 3. Your function my_ID3() should operate in a manner such that after every round of decision making, it will output the attributes and its associated gain, with a message stating “Attribute X with Gain = Y is chosen as the decision attribute”. Once your function completes, it should output the decision tree. The representation of the decision tree is upto you. You can choose either a textual representation or a graphical one; either is fine. (10 marks)

In [41]:
def calculateEntropy(data):
    targetClass = data.keys()[-1]
    print("target Class in entropy : " + targetClass)
    entropyValue = 0
    targetClassUnique = data[targetClass].unique()

    for item in targetClassUnique:
        proportion = data[targetClass].value_counts()[item]/ len (data[targetClass])
        entropyValue += -proportion * np.log2(proportion)

    return entropyValue

In [42]:
def getEntropyAttribute(data,attribute):
    targetClass = data.keys()[-1]
    print("target Class in entropy attribute : " + targetClass)
    targetUniqueValues = data[targetClass].unique()
    
    attributeUniqueValue = data[attribute].unique()
    entropyValue = 0

    for unique in attributeUniqueValue:
        entropy = 0
        for target in targetUniqueValues:
            num = len(data[attribute][data[attribute] == unique][data[targetClass] == target])
            den = len(data[attribute][data[attribute] == unique])

            fraction = num/ (den + eps)
            entropy += -fraction * np.log2(fraction + eps)
        
        fraction2=den/len(data)
        print("FRACTION : " , fraction , " | FRACTION2 : ", fraction2)
        entropyValue += -fraction2 * entropy

    return abs(entropyValue)

In [43]:
def builtClassifier(data):
    informationGain = []

    for key in data.keys()[:-1]:
        print(key, " -> Entropy : ", calculateEntropy(data) , ' -> Info Gain : ' , getEntropyAttribute(data, key) )
        informationGain.append(calculateEntropy(data) - getEntropyAttribute(data, key))

    return data.keys()[:-1][np.argmax(informationGain)] 

In [44]:
def getSubtable(data, feature, value):
    return data[data[feature] == value].reset_index(drop=True)

In [45]:
def my_ID3(data, tree=None):
    targetClass = data.keys()[-1]
    attribute = builtClassifier(data)

    attributeUnique = np.unique(data[attribute])

    if tree is None:
        tree = {}
        tree[attribute] = {}

    for unique in attributeUnique:
        subTable = getSubtable(data, attribute, unique)
        print("UNIQUE VALLUE : " , unique)
        
        targetValues, valueCounts = np.unique(subTable[targetClass], return_counts=True)

        if len(valueCounts) == 1:
            tree[attribute][unique] = targetValues[0]
        else:
            tree[attribute][unique] = my_ID3(subTable)
        
        print("tree : ", tree)
        print()
    return tree

In [46]:
print(data.head())
decisionTree = my_ID3(data)
pprint.pprint(decisionTree)

      Mood  Effort  Score_46.0  Score_69.5  Score_81.5 Output
0    Happy     Low           1           1           1    Yes
1    Happy    High           1           1           1    Yes
2      Sad     Low           1           1           1    Yes
3    Happy     Low           1           1           1    Yes
4  Neutral  Medium           0           1           1     No
target Class in entropy : Output
target Class in entropy attribute : Output
FRACTION :  0.5714285714285714  | FRACTION2 :  0.3684210526315789
FRACTION :  0.42857142857142855  | FRACTION2 :  0.3684210526315789
FRACTION :  0.8  | FRACTION2 :  0.2631578947368421
Mood  -> Entropy :  0.9819407868640977  -> Info Gain :  0.9159386515219116
target Class in entropy : Output
target Class in entropy attribute : Output
FRACTION :  0.5714285714285714  | FRACTION2 :  0.3684210526315789
FRACTION :  0.42857142857142855  | FRACTION2 :  0.3684210526315789
FRACTION :  0.8  | FRACTION2 :  0.2631578947368421
target Class in entropy : Output
